In [1]:
from transformers import AutoProcessor
from transformers import SeamlessM4TModel
import torch
import pandas as pd
df = pd.read_csv("/home/ubuntu/multilingual-model-card/src/dictionary_collection/mturk/whole_set/Chinese_whole.csv")

processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-Large", use_fast=False, force_download=True)
model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-Large")

/home/ubuntu/miniconda3/envs/mmmc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/ubuntu/miniconda3/envs/mmmc/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/ubuntu/miniconda3/envs/mmmc/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_glo

In [6]:
ai_terms_dict = {row['word']: row['gold_human'] for _, row in df.iterrows()}

In [33]:
ai_terms_dict = {"Decision Transformer": "决策Transformer"}

In [4]:
from transformers import LogitsProcessor

seamless_lang_dict = {
    "Arabic": "arb",
    "Chinese": "cmn",
    "English": "eng",
    "French": "fra",
    "Japanese": "jpn",
    "Russian": "rus",
}

class TerminologyAwareLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer, en_text, lang, soft_penalty, seamless_lang_dict, tokens):
        self.tokenizer = tokenizer
        self.en_text = en_text
        self.lang = lang
        self.soft_penalty = soft_penalty
        self.seamless_lang_dict = seamless_lang_dict
        self.tokens = tokens

    def __call__(self, input_ids, scores):
        # print(scores)
        for idx in range(scores.shape[-1]):
            tokens = list({item for sublist in self.tokens for item in sublist})
            if idx in tokens:  # Penalize tokens not in the translation
                scores[:, idx] /= self.soft_penalty  # Apply soft penalty
        # print(scores)
        return scores

def generate_with_constraints(model, tokenizer, ai_terms_dict, en_text, lang, num_beams=5, soft_penalty=0.8):
    # Tokenize the input text
    inputs = tokenizer(en_text, src_lang = seamless_lang_dict['English'], return_tensors="pt")
    # print(inputs)
    
    tokens = []
    terms = []
    for term, translation in ai_terms_dict.items():
        if term in en_text:  # Context-sensitive dictionary match
                token = tokenizer(translation, src_lang=seamless_lang_dict[lang])['input_ids']
                tokens.append(token)
                terms.append(term)
    # print(tokens)
    # print(terms)
    
    logits_processor = TerminologyAwareLogitsProcessor(
        tokenizer=tokenizer,
        en_text=en_text,
        lang=lang,
        soft_penalty=soft_penalty,
        seamless_lang_dict=seamless_lang_dict,
        tokens = tokens,
    )

    # Prepare the beam search decoder
    generated_ids = model.generate(
        **inputs,
        tgt_lang=seamless_lang_dict[lang],
        generate_speech=False,
        # num_beams=num_beams,
        # early_stopping=True,
        # output_scores=True,
        # return_dict_in_generate=True,
        logits_processor=[logits_processor]
    )

    # Decode the output sequence
    output = tokenizer.decode(generated_ids[0].tolist()[0], skip_special_tokens=True)
    return output


In [6]:
input_text = "1: For all the methods, we used 10-fold cross validation (i.e., each fold we have 556 training and 62 test samples) to tune free parameters, e.g., the kernel form and parameters for GPOR and LapSVM. Note that all the alternative methods stack X and Z together into a whole data matrix and ignore their heterogeneous nature."
# translation = generate_with_constraints(model, processor, ai_terms_dict, input_text, lang='Chinese', soft_penalty=0.85)
# print("Translated Text:", translation)


In [7]:
text_inputs = processor(
    text=input_text,
    src_lang=seamless_lang_dict['English'],
    return_tensors='pt'
)
output_tokens = model.generate(**text_inputs, tgt_lang=seamless_lang_dict['Chinese'], generate_speech=False)
processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)

Keyword arguments {'add_special_tokens': False} not recognized.


'1:对于所有方法,我们使用了10倍的交叉验证(即,每倍我们有556个训练和62个测试样本)来调整自由参数,例如,内核形式和GPOR和LapSVM的参数.'